In [1]:
def get_points_from_face_results(face_results):
    positions = []
    for landmark in face_results.multi_face_landmarks[0].landmark:
        positions.append([landmark.x - 0.5, -landmark.y + 0.5, -landmark.z])
    return positions

In [2]:
from pythreejs import *
from IPython.display import display
import time
import random

positions = get_points_from_face_results(face_results)

mask_geometry = BufferGeometry(attributes={
    'position': BufferAttribute(positions),
})

cube_geometry = BoxGeometry(1, 0.5, 1)
cube_material = MeshBasicMaterial(color='red', wireframe=True)
mask_material = PointsMaterial(color='blue', size=0.02, sizeAttenuation=True)

cube = Mesh(geometry=cube_geometry, material=cube_material)
mask = Points(geometry=mask_geometry, material=mask_material)

scene = Scene(children=[cube, mask, AmbientLight(color='#777777')])

camera = PerspectiveCamera(position=[0, 0, 1], aspect=1.5)
# camera.lookAt([10, -10, 10])

renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=600, height=400)

# Display the cube
display(renderer)

# while True:
#     new_positions = np.random.rand(num_points, 3) * 2 - 1
#     # Update the geometry's position attribute
#     mask.geometry.attributes['position'].array = new_positions
#     # Notify the renderer that the geometry has changed
#     mask.geometry.attributes['position'].needsUpdate = True
#     time.sleep(0.001)  # Wait for 1 second
#     renderer.render(scene, camera)

NameError: name 'face_results' is not defined

In [3]:
import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cap = cv2.VideoCapture(0)  # 0 for the default webcam

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame for face and hand landmarks
    face_results = face_mesh.process(rgb_frame)

    if face_results.multi_face_landmarks:
        mask.geometry.attributes['position'].array = get_points_from_face_results(face_results)
        mask.geometry.attributes['position'].needsUpdate = True
        renderer.render(scene, camera)
        # for face_landmarks in face_results.multi_face_landmarks:
        #     mp_drawing.draw_landmarks(
        #         frame,
        #         face_landmarks,
        #         mp_face_mesh.FACEMESH_TESSELATION,
        #         landmark_drawing_spec=None,
        #         connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

    cv2.imshow('MediaPipe Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()